In [1]:
import pandas as pd 
import numpy as np
from geopy.geocoders import Nominatim

from functools import lru_cache

In [2]:
df = pd.read_csv("policekillings.csv")
df.shape

(9081, 38)

In [3]:
# filter null city values
df = df[(df["City"].notnull()) & (df["State"].notnull())]
df.shape

(9072, 38)

In [4]:
# test city conversions
address='Ithaca, NY'
geolocator = Nominatim(user_agent="Your_Name")
location = geolocator.geocode(address, timeout = None)
print(location.address)
print((location.latitude, location.longitude))

Ithaca, Ithaca Town, Tompkins County, New York, United States
(42.4396039, -76.4968019)


In [9]:
# gen coordinates
# memoization table --> geolocator calls limited by api refresh rate
m = {}
bad_adds = []
def gen_coord(row):
    address = row["City"] + ", " + row["State"]
    if address in m:
        return m[address]
    else:
        location = geolocator.geocode(address)
        if location:
            m[address] = (location.latitude, location.longitude)
        else:
            print(address)
            bad_adds.append(address)
            m[address] = None
    return m[address]

df["coordinates"] = df[["City", "State"]].apply(gen_coord, axis = 1)
df.head()

South Chesterfield, VA
Clarkesville, TX
King of Prussi, PA
Wasau, WI
Oveido, FL
Gardernville, NV
Catersville, GA
Constantania, NY
Shrevport, LA
Racoine, WI
Dunelion, FL


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Liberty+Township%2C+MN&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [22]:
len(m)

3661

In [19]:
len(df["City"].unique())

3254

In [30]:
def gen_coord_aux(row):
    address = row["City"] + ", " + row["State"]
    if address in m:
        return m[address]
    else:
        print(address)
        bad_adds.append(address)
        return None 

df["coordinates"] = df.apply(gen_coord_aux, axis = 1)
df.head()

Liberty Township, MN
Maysel, WV
Fairview, AK
Lexington, MS
Roseburg, OR
St. Regis, MT
Salem, CT
Montgomery, IN
Randolph, MA
Mullan, ID
Sardis, GA
Hampden, ME
Paden, OK
Thornton, IL
Meriden, CT
Wilmer, AL
Afton, VA
Millersville, MD
Windsor, CA
Hobe Sound, FL
Bonham, TX
Highland Park, TX
St. Albans, NY
San Jacinto, CA
Mad River, CA
Red Feather Lakes, CO
Waimanalo, HI
Pine, CO
Broad Creek, NC
Sterling, VA
Helena, MT
Crossville, TN
Eden, TX
Salida, CO
Mt. Morris Township, MI
Washington Terrace, UT
Ridgefield, CT
Worden, MT
Sutton-Alpine, AK
Clifton, TX
Williamsburg, VA
Three Way, TX
Eden, TX
Evanston, WY
Uniondale, NY
Uniondale, NY
Nine Mile Falls, WA
Clintonville, OH
Clintonville, OH
Oakville, MO
Stromsburg, NE
Cudahy, CA
Eureka, MO
Richmond, IL
Stringtown, OK
Warren, OR
San Leon, TX
Coeburn, VA
Vernal, UT
Homerville, GA
Melbourne, AR
Frog, TX
Athens, CA
Atascocita, TX
Manchester, IL
Pleasantville, NJ
Longwood, FL
Calumet City, IL
Merritt Island, FL
Crystal Springs, MS
Westfield Boro, PA


,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,...,Encounter Type (DRAFT),Initial Reported Reason for Encounter (DRAFT),Names of Officers Involved (DRAFT),Race of Officers Involved (DRAFT),Known Past Shootings of Officer(s) (DRAFT),Call for Service? (DRAFT),Unnamed: 35,Unnamed: 36,Unnamed: 37,coordinates
0,Name withheld by police,Unknown,Male,Unknown Race,NaN,4/18/21,NaN,Detroit,MI,NaN,...,Part 1 Violent Crime,shooting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(42.3315509, -83.0466403)"
1,Name withheld by police,Unknown,Male,Unknown Race,NaN,4/18/21,NaN,Douglasville,GA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(33.7514966, -84.7477136)"
2,Name withheld by police,Unknown,Male,Unknown Race,NaN,4/18/21,NaN,Fort Worth,TX,NaN,...,Other Non-Violent Offense,carjacking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(32.753177, -97.3327459)"
3,Name withheld by police,Unknown,Male,White,NaN,4/18/21,NaN,Burnsville,MN,NaN,...,Other Non-Violent Offense,carjacking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(44.7670567, -93.2773887)"
4,Name withheld by police,Unknown,Male,Black,NaN,4/17/21,NaN,Winter Haven,FL,NaN,...,Domestic disturbance,Domestic disturbance (armed),NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(28.0222435, -81.7328568)"


In [34]:
len(bad_adds)
print("Before filter" + str(df.shape))
# bad addresses mainly due to a misspell --> ex: Racione, WI instead of Racine, WI
# decided to simply filter out bad values because 167 addresses would be a lot to hard code 
df = df[df['coordinates'].notna()]
print("After filter" + str(df.shape))


Before filter(9072, 39)
After filter(8904, 39)


In [37]:
# now that all rows have coordinate positions convert back to csv for use
df.to_csv("policeCleaned.csv", index = False)

In [38]:
# check csv output
tmp = pd.read_csv("policeCleaned.csv")
tmp.head()

,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,...,Encounter Type (DRAFT),Initial Reported Reason for Encounter (DRAFT),Names of Officers Involved (DRAFT),Race of Officers Involved (DRAFT),Known Past Shootings of Officer(s) (DRAFT),Call for Service? (DRAFT),Unnamed: 35,Unnamed: 36,Unnamed: 37,coordinates
0,Name withheld by police,Unknown,Male,Unknown Race,NaN,4/18/21,NaN,Detroit,MI,NaN,...,Part 1 Violent Crime,shooting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(42.3315509, -83.0466403)"
1,Name withheld by police,Unknown,Male,Unknown Race,NaN,4/18/21,NaN,Douglasville,GA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(33.7514966, -84.7477136)"
2,Name withheld by police,Unknown,Male,Unknown Race,NaN,4/18/21,NaN,Fort Worth,TX,NaN,...,Other Non-Violent Offense,carjacking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(32.753177, -97.3327459)"
3,Name withheld by police,Unknown,Male,White,NaN,4/18/21,NaN,Burnsville,MN,NaN,...,Other Non-Violent Offense,carjacking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(44.7670567, -93.2773887)"
4,Name withheld by police,Unknown,Male,Black,NaN,4/17/21,NaN,Winter Haven,FL,NaN,...,Domestic disturbance,Domestic disturbance (armed),NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(28.0222435, -81.7328568)"
